In [23]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from datetime import datetime,date,timedelta
import pickle

In [2]:
station = 'KCAWATSO86'
staname = 'watsonvil'
now = datetime.now().date()
sdate = datetime(now.year-2,now.month-5,now.day-5).date()
datediff = (now-sdate).days

In [3]:
allwind = pd.read_pickle(f'{staname}_raw_{sdate}_{now-timedelta(days=1)}.pkl')

In [4]:
allwind[::len(allwind)//10]

,Date,Time,Season,Month,Daypart,Temp,Wind,Azimuth,Speed,Gust
0,2021-09-25,12:04 AM,Fall,Sep,Overnight,56.3,South,180.0,0.0,0.5
151,2021-11-03,12:39 PM,Fall,Nov,Midday,76.4,SSW,202.5,0.2,2.0
3,2021-12-12,12:19 AM,Winter,Dec,Overnight,33.3,ESE,112.5,0.3,1.2
147,2022-01-20,12:24 PM,Winter,Jan,Midday,68.4,SSW,202.5,1.3,4.4
214,2022-02-28,5:54 PM,Winter,Feb,Afternoon,68.6,East,90.0,0.0,0.1
108,2022-04-09,9:09 AM,Spring,Apr,Morning,50.6,East,90.0,0.4,1.4
52,2022-05-18,4:24 AM,Spring,May,Overnight,41.4,East,90.0,0.1,1.3
41,2022-06-29,3:29 AM,Summer,Jun,Overnight,50.4,East,90.0,0.0,0.2
228,2022-08-07,7:09 PM,Summer,Aug,Evening,74.8,SE,135.0,0.1,1.1
140,2022-09-15,11:54 AM,Fall,Sep,Midday,72.1,South,180.0,0.1,1.9


In [5]:
wind_wsw = allwind.query('Wind == "WSW"')
wind_ene = allwind.query('Wind == "ENE"')

In [6]:
print(f'wsw = {round(100*len(wind_wsw)/len(allwind),1)}% |  ene = {round(100*len(wind_ene)/len(allwind),1)}%')

wsw = 8.1% |  ene = 3.7%


In [7]:
def dftostrFF(df):
    print(df.reset_index(drop=False).to_string(index=False))
def dftostrTT(df):
    print(df.reset_index(drop=True).to_string(index=True))
def dftostrTF(df):
    print(df.reset_index(drop=True).to_string(index=False))
def dftostrFT(df):
    print(df.reset_index(drop=False).to_string(index=True))

In [8]:
ordir={"North":0.0,
    "NNE":22.5,
    "NE": 45.0,
    "ENE":67.5,
    "East":90.0,
    "ESE":112.5,
    "SE":135.0,
    "SSE":157.5,
    "South":180,
    "SSW":202.5,
    "SW":225.0,
    "WSW":247.5,
    "West":270.0,
    "WNW":292.5,
    "NW":315.0,
    "NNW":337.5}
revdir = [(value, key) for key, value in ordir.items()]
print('mapping of 16 angular wind directions to 16 compass points:\n')
print(pd.DataFrame(revdir,columns=['Azimuth','Compass']).to_string(index=False))
revord = dict(revdir)

mapping of 16 angular wind directions to 16 compass points:

 Azimuth Compass
     0.0   North
    22.5     NNE
    45.0      NE
    67.5     ENE
    90.0    East
   112.5     ESE
   135.0      SE
   157.5     SSE
   180.0   South
   202.5     SSW
   225.0      SW
   247.5     WSW
   270.0    West
   292.5     WNW
   315.0      NW
   337.5     NNW


In [9]:
def ordinalify(df,groupcol,grouporder):
    avgdir = df.groupby(groupcol)['Azimuth'].mean()[grouporder]
    avgint = avgdir.astype(int)
    clsdir = round(avgdir/22.5,0)*22.5
    dford = pd.concat([pd.concat([pd.DataFrame(avgint),pd.DataFrame(clsdir)],axis=1),clsdir.map(revord)],axis=1)
    dford.columns=['Azimuth','Closest','Compass']
    dford = dford.reset_index(drop=False)
    return dford

In [10]:
print("average wind Azimuth by season and daypart:\n")
dfs = allwind.groupby(['Season','Daypart'])['Azimuth'].mean().round(1)
dfs = dfs.reset_index(drop=False)
dfs['Season'] = pd.Categorical(dfs['Season'], ['Winter','Spring','Summer','Fall'])
dfs['Daypart'] = pd.Categorical(dfs['Daypart'], ['Overnight','Morning','Midday','Afternoon','Evening'])
dftostrTF(dfs.sort_values(['Season','Daypart']))

average wind Azimuth by season and daypart:

Season   Daypart  Azimuth
Winter Overnight    103.6
Winter   Morning    109.2
Winter    Midday    187.5
Winter Afternoon    175.0
Winter   Evening    112.6
Spring Overnight    104.9
Spring   Morning    117.8
Spring    Midday    198.8
Spring Afternoon    195.5
Spring   Evening    130.6
Summer Overnight    118.7
Summer   Morning    136.6
Summer    Midday    215.4
Summer Afternoon    221.0
Summer   Evening    170.4
  Fall Overnight    113.4
  Fall   Morning    114.7
  Fall    Midday    203.5
  Fall Afternoon    205.4
  Fall   Evening    143.5


In [11]:
print("average wind speed by season and daypart:\n")
dfs = allwind.groupby(['Season','Daypart'])['Speed'].mean().round(1)
dfs = dfs.reset_index(drop=False)
dfs['Season'] = pd.Categorical(dfs['Season'], ['Winter','Spring','Summer','Fall'])
dfs['Daypart'] = pd.Categorical(dfs['Daypart'], ['Overnight','Morning','Midday','Afternoon','Evening'])
dftostrTF(dfs.sort_values(['Season','Daypart']))

average wind speed by season and daypart:

Season   Daypart  Speed
Winter Overnight    0.4
Winter   Morning    0.5
Winter    Midday    0.8
Winter Afternoon    0.6
Winter   Evening    0.3
Spring Overnight    0.3
Spring   Morning    0.4
Spring    Midday    1.0
Spring Afternoon    1.0
Spring   Evening    0.6
Summer Overnight    0.1
Summer   Morning    0.2
Summer    Midday    0.7
Summer Afternoon    0.7
Summer   Evening    0.4
  Fall Overnight    0.1
  Fall   Morning    0.2
  Fall    Midday    0.4
  Fall Afternoon    0.4
  Fall   Evening    0.1


In [12]:
print("ascending avg wind speed by season and daypart:\n")
dftostrTF(dfs.sort_values('Speed'))

ascending avg wind speed by season and daypart:

Season   Daypart  Speed
  Fall   Evening    0.1
  Fall Overnight    0.1
Summer Overnight    0.1
  Fall   Morning    0.2
Summer   Morning    0.2
Spring Overnight    0.3
Winter   Evening    0.3
Summer   Evening    0.4
  Fall Afternoon    0.4
  Fall    Midday    0.4
Spring   Morning    0.4
Winter Overnight    0.4
Winter   Morning    0.5
Spring   Evening    0.6
Winter Afternoon    0.6
Summer Afternoon    0.7
Summer    Midday    0.7
Winter    Midday    0.8
Spring    Midday    1.0
Spring Afternoon    1.0


In [13]:
print("unweighted average wind azimuth direction by season:\n")
orderszn = ['Winter','Spring','Summer','Fall']
dftostrTF(ordinalify(allwind,'Season',orderszn))
print("\n\nunweighted average wind azimuth direction by month:\n")
ordermth=['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
dftostrTF(ordinalify(allwind,'Month',ordermth))

unweighted average wind azimuth direction by season:

Season  Azimuth  Closest Compass
Winter      132    135.0      SE
Spring      142    135.0      SE
Summer      163    157.5     SSE
  Fall      148    157.5     SSE


unweighted average wind azimuth direction by month:

Month  Azimuth  Closest Compass
  Jan      130    135.0      SE
  Feb      138    135.0      SE
  Mar      140    135.0      SE
  Apr      140    135.0      SE
  May      144    135.0      SE
  Jun      158    157.5     SSE
  Jul      164    157.5     SSE
  Aug      166    157.5     SSE
  Sep      165    157.5     SSE
  Oct      146    157.5     SSE
  Nov      134    135.0      SE
  Dec      128    135.0      SE


In [14]:
def groupwavg(groupcol,grouporder):
    wgtdir = (nnwind['Speed']).multiply(nnwind['Azimuth'])
    dfwgt = pd.concat([nnwind,pd.DataFrame(wgtdir,columns=['Wgtdir'])],axis=1)[[groupcol,'Speed','Wgtdir']]
    dfsum = dfwgt.groupby(groupcol).sum()
    wavgd = (dfsum.Wgtdir/dfsum.Speed)
    wintd = wavgd.astype(int)
    wclsd = round(wavgd/22.5,0)*22.5
    wcomp = wclsd.map(revord)
    dfwavg = pd.concat([pd.concat([wintd,wclsd],axis=1),wcomp],axis=1)
    dfwavg.columns = ['Azimuth','Closest','Compass']
    dfwavg = dfwavg.reset_index(drop=False)
    dfwavg[groupcol] = pd.Categorical(dfwavg[groupcol], grouporder)
    return dfwavg.sort_values(groupcol)

In [15]:
nnwind = allwind[allwind['Azimuth'].notnull()]

In [16]:
print("speed weighted average wind azimuth direction by Season:\n")
dftostrTF(groupwavg('Season',['Winter','Spring','Summer','Fall']))
print("\n\nspeed weighted average wind azimuth direction by Month:\n")
dftostrTF(groupwavg('Month',ordermth))

speed weighted average wind azimuth direction by Season:

Season  Azimuth  Closest Compass
Winter      151    157.5     SSE
Spring      163    157.5     SSE
Summer      199    202.5     SSW
  Fall      167    157.5     SSE


speed weighted average wind azimuth direction by Month:

Month  Azimuth  Closest Compass
  Jan      162    157.5     SSE
  Feb      168    157.5     SSE
  Mar      173    180.0   South
  Apr      158    157.5     SSE
  May      160    157.5     SSE
  Jun      187    180.0   South
  Jul      200    202.5     SSW
  Aug      217    225.0      SW
  Sep      178    180.0   South
  Oct      164    157.5     SSE
  Nov      162    157.5     SSE
  Dec      132    135.0      SE


In [17]:
print("difference in monthly average azimuth without vs with wind speed as weights:\n")
dfdiff=pd.concat([pd.Series(ordermth),(ordinalify(allwind,'Month',ordermth)).Azimuth.subtract((groupwavg('Month',ordermth)).Azimuth)],axis=1)
dfdiff.columns=['Month','Azimuth']
dftostrTF(dfdiff)
print("\n\ndifference in seasonal average azimuth without vs with wind speed as weights:\n")
dfdiff=pd.concat([pd.Series(orderszn),(ordinalify(allwind,'Season',orderszn)).Azimuth.subtract((groupwavg('Season',orderszn)).Azimuth)],axis=1)
dfdiff.columns=['Season','Azimuth']
dftostrTF(dfdiff)

difference in monthly average azimuth without vs with wind speed as weights:

Month  Azimuth
  Jan      -28
  Feb      -79
  Mar        8
  Apr      -28
  May      -18
  Jun      -42
  Jul      -23
  Aug       -7
  Sep        5
  Oct      -16
  Nov      -30
  Dec      -50


difference in seasonal average azimuth without vs with wind speed as weights:

Season  Azimuth
Winter      -35
Spring      -21
Summer      -36
  Fall       -3


In [18]:
list(ordir.keys())

['North',
 'NNE',
 'NE',
 'ENE',
 'East',
 'ESE',
 'SE',
 'SSE',
 'South',
 'SSW',
 'SW',
 'WSW',
 'West',
 'WNW',
 'NW',
 'NNW']

In [19]:
allwind.query("Wind == ''")

,Date,Time,Season,Month,Daypart,Temp,Wind,Azimuth,Speed,Gust
183,2021-09-25,3:19 PM,Fall,Sep,Afternoon,72.4,,NaN,0.5,2.5
34,2021-09-28,2:54 AM,Fall,Sep,Overnight,56.0,,NaN,0.9,3.3
218,2021-09-28,6:14 PM,Fall,Sep,Evening,71.0,,NaN,0.6,2.8
155,2021-10-01,12:59 PM,Fall,Oct,Midday,90.6,,NaN,0.5,2.6
169,2021-10-08,2:09 PM,Fall,Oct,Afternoon,70.1,,NaN,1.2,4.7
...,...,...,...,...,...,...,...,...,...,...
59,2022-09-22,1:14 PM,Fall,Sep,Midday,82.7,,NaN,0.5,2.4
269,2022-09-28,10:29 PM,Fall,Sep,Overnight,64.2,,NaN,0.1,0.5
132,2022-10-08,11:09 AM,Fall,Oct,Midday,71.2,,NaN,0.1,1.3
239,2022-10-09,7:59 PM,Fall,Oct,Evening,57.4,,NaN,0.0,0.6


In [20]:
allwind.query("Azimuth.isnull()")

,Date,Time,Season,Month,Daypart,Temp,Wind,Azimuth,Speed,Gust
183,2021-09-25,3:19 PM,Fall,Sep,Afternoon,72.4,,NaN,0.5,2.5
34,2021-09-28,2:54 AM,Fall,Sep,Overnight,56.0,,NaN,0.9,3.3
218,2021-09-28,6:14 PM,Fall,Sep,Evening,71.0,,NaN,0.6,2.8
155,2021-10-01,12:59 PM,Fall,Oct,Midday,90.6,,NaN,0.5,2.6
169,2021-10-08,2:09 PM,Fall,Oct,Afternoon,70.1,,NaN,1.2,4.7
...,...,...,...,...,...,...,...,...,...,...
59,2022-09-22,1:14 PM,Fall,Sep,Midday,82.7,,NaN,0.5,2.4
269,2022-09-28,10:29 PM,Fall,Sep,Overnight,64.2,,NaN,0.1,0.5
132,2022-10-08,11:09 AM,Fall,Oct,Midday,71.2,,NaN,0.1,1.3
239,2022-10-09,7:59 PM,Fall,Oct,Evening,57.4,,NaN,0.0,0.6


In [22]:
print("average wind speed by direction:\n")
dfs = allwind.groupby('Wind')['Speed'].mean().round(1)
dfs = dfs.reset_index(drop=False)
dfs['Wind'] = pd.Categorical(dfs['Wind'], list(ordir.keys()))
dftostrTF(dfs.sort_values('Wind'))

print("ascending avg wind speed by direction:\n")
dftostrTF(dfs.sort_values('Speed'))

average wind speed by direction:

 Wind  Speed
North    0.5
  NNE    0.5
   NE    0.7
  ENE    0.4
 East    0.3
  ESE    0.3
   SE    0.4
  SSE    0.3
South    0.3
  SSW    0.5
   SW    0.7
  WSW    0.7
 West    0.5
  WNW    0.7
   NW    0.8
  NNW    0.7
  NaN    0.8
ascending avg wind speed by direction:

 Wind  Speed
  ESE    0.3
 East    0.3
South    0.3
  SSE    0.3
  ENE    0.4
   SE    0.4
North    0.5
  SSW    0.5
 West    0.5
  NNE    0.5
  NNW    0.7
   SW    0.7
   NE    0.7
  WNW    0.7
  WSW    0.7
   NW    0.8
  NaN    0.8
